# semdoc
#### Genera ficheros HTML de documentación de un modelo semántico publicado en el servicio de Power BI / Fabric

![Ejemplo de las páginas HTML generadas ](resources/dataXbi-fabric-semdoc.png)

Este notebook es una adaptación a Fabric del comando semdoc de la herramienta **pbicmd** (https://github.com/dataxbi/pbicmd)

- Utiliza la librería [semantic link](https://learn.microsoft.com/en-us/fabric/data-science/semantic-link-overview) por lo que se recomienda utilizarlo en un área de trabajo o en un entorno con Spark 4.3 o superior, que ya trae semantic link instalado.
- También se recomienda vincular un Lakehouse al notebook y crear una subcarpeta para almacenar los ficheros HTML.
- Se recomienda utilizar [OneLake file explorer](https://learn.microsoft.com/en-us/fabric/onelake/onelake-file-explorer) para visualizar localmente los ficheros HTML generados.

Semantic link:
- De semantic link se utiliza la clase PowerBIRestClient para conectarse con la API REST de Power BI y ejecutar los comandos INFO de DAX.
- No se utilizan funciones como list_tables, list_measures, etc. porque dichas funciones utilizan una conexión XMLA, por lo que no se pueden usar en modelos alojados en áreas de trabajo con licencia Pro.
- No se están aprovechando los *enum* de la librería Microsoft.AnalysisServices.Tabular, que está incluida en semantic link, sino que se están usando los mismos *enum* definidos en el comando semdoc de pbicmd. Esto ha sido por una cuestión de comodidad para no tener que hacer muchos cambios en el resto del código. Pero no descartamos usarlos en el futuro.

Librería NotebookUtils
- Otra diferencia de esta notebook con **pbicmd** es que aquí se utiliza la librería de Microsoft [NotebookUtils](https://learn.microsoft.com/en-us/fabric/data-engineering/microsoft-spark-utilities), llamada antes MsSparkUtils, para manejar la carpeta donde se guardan los ficheros HTML.

In [1]:
# Esta es una celda de parámetros, por si se quiere utilizar este notebook desde una canalización.
# Cambiar aquí los valores del área de trabajo, el modelo semántico y la ruta a la carpeta donde se guardarán los ficheros HTML.

WORKSPACE_ID = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
DATASET_ID = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
HTML_FOLDER = f"/lakehouse/default/Files/semdoc_html/{DATASET_ID}"

StatementMeta(, 0adcee9e-0c7b-4301-8d32-7386cc155280, 3, Finished, Available, Finished)

In [2]:
from datetime import datetime
from enum import Enum

from jinja2 import Environment, DictLoader
from notebookutils import mssparkutils
import pandas as pd
import sempy.fabric as fabric

StatementMeta(, 0adcee9e-0c7b-4301-8d32-7386cc155280, 4, Finished, Available, Finished)

In [3]:
# Estos enum son un poco redundantes porque semantic link ya incluye la librería Microsoft.AnalysisServices.Tabular
# pero por ahora se están usando exactamente como están en el comando semdoc de pbicmd para no tener que cambiar otras partes del código

class DataType(Enum):
    """Este enum es una copia del enum DataType definido en la librería .NET Microsoft.AnalysisServices.Tabular
    https://learn.microsoft.com/en-us/dotnet/api/microsoft.analysisservices.tabular.datatype?view=analysisservices-dotnet
    """

    AUTOMATIC = 1  # Internal only.
    BINARY = 17
    BOOLEAN = 11
    DATE_TIME = 9
    DECIMAL = 10
    DOUBLE = 8
    INT64 = 6
    STRING = 2
    UNKNOWN = 19  # Initial value of a newly created column, replaced with an actual value after saving a Column to the Server.
    VARIANT = 20  # A measure with varying data type.


class ColumnType(Enum):
    """Este enum es una copia del enum ColumnType definido en la librería .NET Microsoft.AnalysisServices.Tabular
    https://learn.microsoft.com/en-us/dotnet/api/microsoft.analysisservices.tabular.columntype?view=analysisservices-dotnet
    """

    CALCULATED = 2  # The contents of this column are computed by using an expression after the Data columns have been populated.
    CALCULATED_TABLE_COLUMN = 4  # The column exists in a calculated table, where the table and its columns are based on a calculated expression.
    DATA = 1  # The contents of this column come from a DataSource.
    ROW_NUMBER = 3  # This column is automatically added by the Server to every table.


class RelationshipEndCardinality(Enum):
    """Este enum es una copia del enum RelationshipEndCardinality definido en la librería .NET Microsoft.AnalysisServices.Tabular
    https://learn.microsoft.com/en-us/dotnet/api/microsoft.analysisservices.tabular.relationshipendcardinality?view=analysisservices-dotnet
    """

    MANY = 2  # Specifies the 'many' side of a one-to-many relationship.
    NONE = 0  # The relationship is unspecified.
    ONE = 1  # Specifies the 'one' side of a one-to-one or one-to-many relationship.


class CrossFilteringBehavior(Enum):
    """Este enum es una copia del enum CrossFilteringBehavior definido en la librería .NET Microsoft.AnalysisServices.Tabular
    https://learn.microsoft.com/en-us/dotnet/api/microsoft.analysisservices.tabular.crossfilteringbehavior?view=analysisservices-dotnet
    """

    AUTOMATIC = 3  # The engine will analyze the relationships and choose one of the behaviors by using heuristics.
    BOTHDIRECTIONS = 2  # Filters on either end of the relationship will automatically filter the other table.
    ONEDIRECTION = 1  # The rows selected in the 'To' end of the relationship will automatically filter scans of the table in the 'From' end of the relationship.


class AggregateFunction(Enum):
    """Este enum es una copia del enum AggregateFunction definido en la librería .NET Microsoft.AnalysisServices.Tabular
    https://learn.microsoft.com/en-us/dotnet/api/microsoft.analysisservices.tabular.aggregatefunction?view=analysisservices-dotnet
    """

    AVERAGE = 7  # Calculates the average of values for all non-empty child members.
    COUNT = 6  # Returns the rows count in the table.
    DEFAULT = 1  # The default aggregation is Sum for numeric columns. Otherwise the default is None.
    DISTINCTCOUNT = 8  # Returns the count of all unique child members.
    MAX = 5  # Returns the highest value for all child members.
    MIN = 4  # Returns the lowest value for all child members.
    NONE = 2  # Leaves the aggregate function unspecified.
    SUM = 3  # Calculates the sum of values contained in the column. This is the default aggregation function.


StatementMeta(, 0adcee9e-0c7b-4301-8d32-7386cc155280, 5, Finished, Available, Finished)

In [4]:
# Estas son las funciones que obtienen los metadatos del modelo semántico.
# Se utiliza la clase PowerBIRestClient de semantic link para garantizar que se haga una conexión a la API REST de Power BI y por lo tanto se puedan documentar modelos alojados en áreas de trabajo con licencia Pro.
# A diferencia de como se implementó en pbicmd, no hace falta preocuparse del token de autenticación, porque semantic link se encarga.

def load_dax_result_to_dataframe(dax_result):
    """Crea un DataFrame pandas con el contenido de la tabla con la respuesta a la consulta DAX.
    Retorna el DataFrame.
    """
    rows = dax_result["results"][0]["tables"][0]["rows"]
    # En el JSON que retorna la API de Power BI, puede ser que no todas las filas tengan las mismas columnas,
    # y json_normalize se encarga de revisar todo el JSON y crear todas las columnas, llenando con NaN las filas que no tengan alguna columna.
    return pd.json_normalize(rows)


def execute_dax_to_dataframe(workspace_id, dataset_id, dax_query):
    """Ejecuta una consulta DAX contra un modelo semántico y devuelve el resultado en un DataFrame pandas."""
    pbi = fabric.PowerBIRestClient()
    r = pbi.post(
        f"/v1.0/myorg/groups/{workspace_id}/datasets/{dataset_id}/executeQueries",
        json = {
            "queries": [{"query": f"{dax_query}"}],
            "serializerSettings": {"includeNulls": True},
        })

    df = load_dax_result_to_dataframe(r.json())

    if not df.empty:
        # Quitando los corchetes de los nombres de las columnas
        df.columns = df.columns.str.replace(r"[\[\]]", "", regex=True)

    return df

def get_dataset_info(workspace_id, dataset_id):
    """Devuelve un diccionario con la información de un modelo semántico, utilizando la API REST de Power BI."""
    pbi = fabric.PowerBIRestClient()
    r = pbi.get(f"/v1.0/myorg/groups/{workspace_id}/datasets/{dataset_id}")
    return r.json()


def get_model_info(workspace_id, dataset_id):
    """Devuelve un diccionario con la información de un modelo semántico, utilizando la API REST de Power BI."""
    dax_query = "EVALUATE INFO.MODEL()"
    model = execute_dax_to_dataframe(workspace_id, dataset_id, dax_query)

    model["Description"] = model["Description"].apply(
        lambda v: v if pd.notna(v) else ""
    )

    return model.iloc[0].to_dict()

def get_model_tables(workspace_id, dataset_id):
    """Devuelve un DataFrame pandas con la lista de tablas del modelo."""
    dax_query = "EVALUATE INFO.TABLES()"
    tables = execute_dax_to_dataframe(workspace_id, dataset_id, dax_query)

    tables["Description"] = tables["Description"].apply(
        lambda v: v if pd.notna(v) else ""
    )

    return tables


def get_model_columns(workspace_id, dataset_id):
    """Devuelve un DataFrame pandas con la lista de columnas del modelo."""
    dax_query = "EVALUATE INFO.COLUMNS()"
    columns = execute_dax_to_dataframe(workspace_id, dataset_id, dax_query)

    # Asegurando que las columnas por las que se van a hacer JOINs tengan tipos compatibles
    columns["ID"] = columns["ID"].astype("Int64")
    columns["SortByColumnID"] = columns["SortByColumnID"].astype("Int64")

    # Aplicando los Enum de TOM
    columns["Type"] = columns["Type"].apply(ColumnType)
    columns["ExplicitDataType"] = columns["ExplicitDataType"].apply(DataType)
    columns["InferredDataType"] = columns["InferredDataType"].apply(DataType)
    columns["SummarizeBy"] = columns["SummarizeBy"].apply(AggregateFunction)    

    # Decidiendo cual es el nombre de la columna
    columns["Name"] = columns.apply(
        lambda c: (
            c["ExplicitName"] if pd.notna(c["ExplicitName"]) else c["InferredName"]
        ),
        axis=1,
    )

    # JOIN con las misma tabla para agregar una columna con el nombre de la clumna SortByColumn
    columns_sort_by = columns[["ID", "Name"]]
    columns_sort_by.columns = ["SortByColumnID", "SortByColumnName"]
    columns = pd.merge(columns, columns_sort_by, how="left", on="SortByColumnID")
    columns["SortByColumnName"] = columns["SortByColumnName"].apply(
        lambda v: v if pd.notna(v) else ""
    )

    # Sustituyendo nan por un texto vacío
    columns["Description"] = columns["Description"].apply(
        lambda v: v if pd.notna(v) else ""
    )
    columns["FormatString"] = columns["FormatString"].apply(
        lambda v: v if pd.notna(v) else ""
    )

    return columns


def get_model_measures(workspace_id, dataset_id):
    """Devuelve un DataFrame de pandas con la lista de medidas del modelo."""
    dax_query = "EVALUATE INFO.MEASURES()"
    measures = execute_dax_to_dataframe(workspace_id, dataset_id, dax_query)

    measures["DataType"] = measures["DataType"].apply(DataType)

    measures["Description"] = measures["Description"].apply(
        lambda v: v if pd.notna(v) else ""
    )
    measures["FormatString"] = measures["FormatString"].apply(
        lambda v: v if pd.notna(v) else ""
    )

    return measures


def get_model_relationships(workspace_id, dataset_id, tables: pd.DataFrame):
    """Devuelve un DataFrame pandas con las relaciones del modelo."""
    dax_query = "EVALUATE INFO.RELATIONSHIPS()"
    relationships = execute_dax_to_dataframe(workspace_id, dataset_id, dax_query)

    tables_rel_from = tables[["ID", "Name"]]
    tables_rel_from.columns = ["FromTableID", "FromTableName"]
    relationships = pd.merge(
        relationships, tables_rel_from, how="left", on="FromTableID"
    )

    tables_rel_to = tables[["ID", "Name"]]
    tables_rel_to.columns = ["ToTableID", "ToTableName"]
    relationships = pd.merge(relationships, tables_rel_to, how="left", on="ToTableID")

    return relationships


def get_model_calculation_groups(workspace_id, dataset_id):
    """Devuelve un DataFrame pandas con los grupos de cálculo."""
    dax_query = "EVALUATE INFO.CALCULATIONGROUPS()"
    cg = execute_dax_to_dataframe(workspace_id, dataset_id, dax_query)

    if not cg.empty:
        cg["Description"] = cg["Description"].apply(lambda v: v if pd.notna(v) else "")

    return cg


def get_model_calculation_items(workspace_id, dataset_id):
    """Devuelve un DataFrame pandas con los calculation items."""
    dax_query = "EVALUATE INFO.CALCULATIONITEMS()"
    ci = execute_dax_to_dataframe(workspace_id, dataset_id, dax_query)

    if not ci.empty:
        ci["Description"] = ci["Description"].apply(lambda v: v if pd.notna(v) else "")

    return ci

StatementMeta(, 0adcee9e-0c7b-4301-8d32-7386cc155280, 6, Finished, Available, Finished)

In [5]:
def get_semantic_model(workspace_id, dataset_id):
    """Devuelve un diccionario con información sobre un modelo semántico publicado en el servicio de Power BI."""
    dataset_info = get_dataset_info(workspace_id, dataset_id)
    model_info = get_model_info(workspace_id, dataset_id)
    tables = get_model_tables(workspace_id, dataset_id)
    columns = get_model_columns(workspace_id, dataset_id)
    measures = get_model_measures(workspace_id, dataset_id)
    relationships = get_model_relationships(workspace_id, dataset_id, tables)
    calculation_groups = get_model_calculation_groups(workspace_id, dataset_id)
    calculation_items = get_model_calculation_items(workspace_id, dataset_id)

    return {
        "generation_time": datetime.now().replace(microsecond=0).isoformat(),
        "dataset_info": dataset_info,
        "model_info": model_info,
        "tables": tables,
        "columns": columns,
        "measures": measures,
        "relationships": relationships,
        "calculation_groups": calculation_groups,
        "calculation_items": calculation_items,
    }


StatementMeta(, 0adcee9e-0c7b-4301-8d32-7386cc155280, 7, Finished, Available, Finished)

In [6]:
def generate_mermaid_relationships(tables, relationships, show_disconnected_tables = True):
    """Genera el código mermaid para dibujar un diagrama que represente las relaciones entre las tablas del modelo."""
    tables_ids_in_relationships = list(
        set(relationships["FromTableID"]) | set(relationships["ToTableID"])
    )
    tables_in_relationships = pd.DataFrame(tables_ids_in_relationships, columns=["ID"])
    tables_in_relationships = tables_in_relationships.merge(
        tables[["ID", "Name"]], on="ID"
    )

    disconnected_tables = tables[~tables["ID"].isin(tables_ids_in_relationships)]
    disconnected_tables = disconnected_tables[["ID", "Name"]]

    mermaid_diagram = ""

    for _, r in tables_in_relationships.iterrows():
        mermaid_diagram += f'T{r["ID"]}[{r["Name"]}]\n'

    for _, r in relationships.iterrows():
        left_table = f'T{r["ToTableID"]}'
        right_table = f'T{r["FromTableID"]}'

        is_active = r["IsActive"]
        is_both_directions = (
            CrossFilteringBehavior(r["CrossFilteringBehavior"]) == CrossFilteringBehavior.BOTHDIRECTIONS
        )
        arrow = ""
        if is_both_directions:
            arrow += "<"
        if is_active:
            arrow += "-->"
        else:
            arrow += "-.->"

        left_cardinality = RelationshipEndCardinality(r["ToCardinality"])
        right_cardinality = RelationshipEndCardinality(r["FromCardinality"])
        cardinality = "|"
        if left_cardinality == RelationshipEndCardinality.ONE:
            cardinality += "1.."
        else:
            cardinality += "*.."
        if right_cardinality == RelationshipEndCardinality.ONE:
            cardinality += "1"
        else:
            cardinality += "*"
        cardinality += "|"

        mermaid_diagram += f"{left_table} {arrow} {cardinality} {right_table} \n"

    if show_disconnected_tables:
        for _, r in disconnected_tables.iterrows():
            mermaid_diagram += f'T{r["ID"]}[{r["Name"]}]\n'

    for _, r in tables_in_relationships.iterrows():
        mermaid_diagram += f'click T{r["ID"]} "table_{r["ID"]}.html"\n'

    if show_disconnected_tables:
        for _, r in disconnected_tables.iterrows():
            mermaid_diagram += f'click T{r["ID"]} "table_{r["ID"]}.html"\n'

    # Si no hay datos en el diagrama, devolver una texto vacío
    if mermaid_diagram == "":
        return ""

    # Si hay datos, comenzar el texto indicando el tipo de gráfico que debe dibujar mermaid
    mermaid_diagram = "graph LR\n" + mermaid_diagram

    return mermaid_diagram

StatementMeta(, 0adcee9e-0c7b-4301-8d32-7386cc155280, 8, Finished, Available, Finished)

In [7]:
def generate_table_page(
    table_id,
    jinja_environment,
    jinja_template,
    html_file_path,
    semantic_model
):
    """Genera una página HTML con información sobre una tabla del modelo semántico.
    Utiliza la plantilla de Jira table.html.

    """

    dataset_info = semantic_model["dataset_info"]
    tables = semantic_model["tables"]
    columns = semantic_model["columns"]
    measures = semantic_model["measures"]
    relationships = semantic_model["relationships"]
    calculation_groups = semantic_model["calculation_groups"]
    calculation_items = semantic_model["calculation_items"]

    # Datos de la tabla
    t = tables[tables["ID"] == table_id].iloc[0].to_dict()
    table_name = t["Name"]
    table_visibility = "Oculta" if t["IsHidden"] else ""
    table_description = t["Description"]

    # Columnas de la tabla
    columns = columns[columns["TableID"] == table_id]
    table_columns = []
    for _, c in columns.iterrows():
        # No tener en cuenta la columna Row_Number
        if c["Type"] == ColumnType.ROW_NUMBER:
            continue

        tc = {}
        tc["name"] = c["Name"]
        tc["visibility"] = "Oculta" if c["IsHidden"] else ""
        tc["data_type"] = (
            c["InferredDataType"].name
            if c["InferredDataType"] != DataType.UNKNOWN
            else c["ExplicitDataType"].name
        )
        tc["format_string"] = ";<br>".join(str(c["FormatString"]).split(";"))
        tc["summarize_by"] = (
            c["SummarizeBy"].name
            if c["SummarizeBy"] != AggregateFunction.NONE
            else ""
        )
        tc["sort_by"] = c["SortByColumnName"]
        tc["description"] = c["Description"]
        table_columns.append(tc)

    # Medidas de la tabla
    measures = measures[measures["TableID"] == table_id]
    table_measures = []
    for _, m in measures.iterrows():
        tm = {}
        tm["name"] = m["Name"]
        tm["visibility"] = "Oculta" if m["IsHidden"] else ""
        tm["data_type"] = m["DataType"].name
        tm["format_string"] = ";<br>".join(str(m["FormatString"]).split(";"))
        tm["expression"] = m["Expression"]
        tm["description"] = m["Description"]
        table_measures.append(tm)

    # Grupo de cálculo asociado a esta tabla, si hay alguno

    table_calculation_items = []
    if not calculation_groups.empty:
        calculation_groups = calculation_groups[
            calculation_groups["TableID"] == table_id
        ]

        if len(calculation_groups) > 0:
            cg = calculation_groups.iloc[0].to_dict()
            calculation_items = calculation_items[
                calculation_items["CalculationGroupID"] == cg["ID"]
            ]
            for _, ci in calculation_items.iterrows():
                tci = {}
                tci["name"] = ci["Name"]
                tci["expression"] = ci["Expression"]
                tci["ordinal"] = ci["Ordinal"]
                tci["description"] = ci["Description"]
                table_calculation_items.append(tci)

    # Filtra las relaciones que corresponden a table_id.
    relationships = relationships[
        (relationships["FromTableID"] == table_id)
        | (relationships["ToTableID"] == table_id)
    ]
    # Dibuja el diagrama de relaciones con esta tabla
    mermaid_diagram = generate_mermaid_relationships(
        tables, relationships, show_disconnected_tables=False
    )

    # Creando la página HTML a partir de la plantilla

    template = jinja_environment.get_template(jinja_template)

    html_content = template.render(
        generation_time=semantic_model["generation_time"],
        dataset_info=dataset_info,
        table_name=table_name,
        table_visibility=table_visibility,
        table_description=table_description,
        table_columns=table_columns,
        table_measures=table_measures,
        table_calculation_items=table_calculation_items,
        mermaid_diagram_relationships=mermaid_diagram,
    )
    with open(html_file_path, mode="w", encoding="utf-8") as f:
        f.write(html_content)


StatementMeta(, 0adcee9e-0c7b-4301-8d32-7386cc155280, 9, Finished, Available, Finished)

In [8]:
def generate_model_page(
    jinja_environment,
    jinja_template,
    html_file_path,
    semantic_model
):
    """Crea una página HTML con información sobre el modelo semántico, incluyendo un diagrama mermaid con las relaciones entre las tablas.
    Utiliza la plantilla de Jira model.html.
    """

    dataset_info = semantic_model["dataset_info"]
    model_info = semantic_model["model_info"]
    tables = semantic_model["tables"]
    relationships = semantic_model["relationships"]

    mermaid_diagram = generate_mermaid_relationships(tables, relationships)
    template = jinja_environment.get_template(jinja_template)

    html_content = template.render(
        generation_time=semantic_model["generation_time"],
        dataset_info=dataset_info,
        model_info=model_info,
        mermaid_diagram=mermaid_diagram,
    )
    with open(html_file_path, mode="w", encoding="utf-8") as f:
        f.write(html_content)

StatementMeta(, 0adcee9e-0c7b-4301-8d32-7386cc155280, 10, Finished, Available, Finished)

In [9]:
# Este diccionario contiene las plantillas de Jinja para poder generar las páginas HTML.
# Se definen aquí mimso en lugar de en ficheros separados, para que todo esté contendio dentro del mismo notebook

jinja_templates = {
    "base.html": """
<!DOCTYPE html>
<html lang="en">
  <head>
    {% block head %}
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <meta name="color-scheme" content="light dark" />
    <link
      rel="stylesheet"
      href="https://cdn.jsdelivr.net/npm/@picocss/pico@2/css/pico.min.css"
    />
    <title>{% block title %}pbicmd semdoc{% endblock %}</title>
    {% endblock %}
  </head>
  <body>
    <header class="container">
        <hgroup>
            <h5 style="color:grey;">DOCUMENTACIÓN DE UN MODELO SEMÁNTICO DE POWER BI</h5>
            <span style="color:grey;">Fecha de generación: <strong>{{ generation_time }}</strong></span>
        </hgroup>
        <hr>
    </header>
    <main class="container">
    {% block content %}
    {% endblock %}
    </main>
    <footer class="container">  
        <hr>
        <small style="color:grey;">Esta documentación fue generada con la herramienta <a href="https://github.com/dataxbi/pbicmd" class="secondary">pbicmd</a></small>
    </footer>
    {% block script %}
    <script type="module">
      import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs";
      const config = {
        startOnLoad: true,
        flowchart: { useMaxWidth: true },
        securityLevel: "loose",
      };
      mermaid.initialize(config);
    </script>
    {% endblock %}
  </body>
</html>
""",
    "model.html": """
{% extends "base.html" %}
{% block title %}{{ dataset_info.name }}{% endblock %}
{% block content %}
<hgroup>
  <h3>{{ dataset_info.name }}</h3>
  <p>{{ model_info.Description }}</p>
</hgroup>


<div><small style="color:grey;">Haga clic sobre una tabla para ver más información.</small></div>
<pre class="mermaid">
{{ mermaid_diagram }}
</pre>
{% endblock %}
""",
    "table.html": """
{% extends "base.html" %}
{% block title %}Tabla: {{table_name}}{% endblock %}
{% block content %}
<hgroup>
  <h5><a href="model.html" style="text-decoration: none;">🡄</a> {{ dataset_info.name }}</h5>
  <h3>Tabla: {{table_name}}</h3>
  {% if table_visibility %}
  <h6>Visibilidad: {{table_visibility}}</h6>
  {% endif %}
  <p>
    {{table_description}}
  </p>
</hgroup>

<hr>
{% if table_columns %}
<a href="#columns">Columnas</a> 
{% endif %}
{% if table_measures %}
| <a href="#measures">Medidas</a>
{% endif %}
{% if table_calculation_items %}
| <a href="#calculation_items">Grupo de cálculo</a>
{% endif %}
{% if mermaid_diagram_relationships %}
| <a href="#relationships">Relaciones con otras tablas</a>
{% endif %}
<hr>

{% if table_columns %}
<h5 id="columns"><a href="#" style="text-decoration: none;">🡅</a> Columnas</h5>
<div class="overflow-auto"></div>
<table>
  <thead>
    <tr>
      <th>Columna</th>
      <th>Visibilidad</th>
      <th>Tipo</th>
      <th>Formato</th>
      <th>Resumen</th>
      <th>Ordenar por</th>
      <th>Descripción</th>
    </tr>  
  </thead>
  <tbody>
  {% for c in table_columns %}
  <tr>
    <td>{{ c.name }}</td>
    <td>{{ c.visibility }}</td>
    <td>{{ c.data_type }}</td>
    <td><small style="font-family:monoespace;">{{ c.format_string }}</small></td>
    <td>{{ c.summarize_by }}</td>
    <td>{{ c.sort_by }}</td>
    <td>
      {{ c.description }}
    </td>
  </tr>
  {% endfor %}
</tbody>
</table>
</div>
{% endif %}

{% if table_measures %}
<h5 id="measures"><a href="#" style="text-decoration: none;">🡅</a> Medidas</h5>
<div class="overflow-auto">
<table>
  <thead>
    <tr>
      <th>Medida</th>
      <th>Carpeta</th>
      <th>Visibilidad</th>
      <th>Tipo</th>
      <th>Formato</th>
      <th>Expresión DAX</th>
      <th>Descripción</th>
    </tr>
  </thead>
  <tbody>
    {% for m in table_measures %}
    <tr>
      <td><span id="mname{{loop.index}}">{{ m.name }}</span></td>
      <td>{{ m.display_folder }}</td>
      <td>{{ m.visibility }}</td>
      <td>{{ m.data_type }}</td>
      <td><small style="font-family:monoespace;">{{ m.format_string }}</small></td>
      <td>
        {% if m.expression %}
          <code onclick="openMeasureDialog('mname{{loop.index}}','mexp{{loop.index}}')" title="Haga clic para mostrar todo el código de la medida" style="cursor:pointer;">{{m.expression|truncate(100,True)}}</code>
          <pre id="mexp{{loop.index}}" style="display:none;">
            {{ m.expression }}
          </pre>
        {% endif %}
      </td>
      <td>
        {{ m.description }}
      </td>
    </tr>
    {% endfor %}
</tbody>
</table>
</div>
{% endif %}

{% if table_calculation_items %}
<h5 id="calculation_items"><a href="#" style="text-decoration: none;">🡅</a> Grupo de cálculo</h5>
<div class="overflow-auto"></div>
<table>
  <thead>
    <tr>
      <th>Item</th>
      <th>Expresión DAX</th>
      <th>Orden</th>
      <th>Descripción</th>
    </tr>
  </thead>
  <tbody>
  {% for ci in table_calculation_items %}
    <tr>
      <td><span id="ciname{{loop.index}}">{{ ci.name }}</span></td>
      <td>
        {% if ci.expression %}
          <pre onclick="openMeasureDialog('ciname{{loop.index}}','ciexp{{loop.index}}')" title="Haga clic para mostrar todo el código DAX" style="cursor:pointer;">{{ci.expression|truncate(300,True)}}</pre>
          <pre id="ciexp{{loop.index}}" style="display:none;">
            {{ ci.expression }}
          </pre>
        {% endif %}
      </td>
      <td>{{ ci.ordinal }}</td>
      <td>
        {{ ci.description }}
      </td>
    </tr>
  {% endfor %}
  </tbody>
</table>
</div>
{% endif %}

{% if mermaid_diagram_relationships %}
<h5 id="relationships"><a href="#" style="text-decoration: none;">🡅</a> Relaciones con otras tablas</h5>
<pre class="mermaid">
  {{ mermaid_diagram_relationships }}
</pre>
{% endif %}

{% endblock %}
{% block script %}
{{ super() }}
<script>
  function openMeasureDialog(measureNameId, measureExpressionId) {
    closeMeasureDialog();
    const measureName = document.querySelector('#' + measureNameId).innerText;
    const measureExpression = document.querySelector('#' + measureExpressionId).innerText;
    const dialog = document.createElement('dialog');
    dialog.setAttribute('id','measureDialog')
    dialog.setAttribute('open', '');
    dialog.innerHTML = `
      <article>
        <header>
          <button aria-label="Close" rel="prev" onclick="closeMeasureDialog()"></button>
          <p>
            <strong>${measureName}</strong>
          </p>
        </header>
        <pre style="max-width:100%;max-height:500px;overflow:auto;">${measureExpression}</pre>
      </article>
    `;    
    document.body.appendChild(dialog);
  }

  function closeMeasureDialog() {
      const dialog = document.querySelector('#measureDialog');
      if (dialog) {
          dialog.remove();
      }
  }
</script>

{% endblock %}

""",
}


StatementMeta(, 0adcee9e-0c7b-4301-8d32-7386cc155280, 11, Finished, Available, Finished)

In [16]:
# En esta función se está utilizando la librería NotebookUtils, antes MsSparkUtils, en lugar de la libreía os como se hace en pbicmd.

def prepare_output_folder(output):
    """Crea la carpeta de salida, si no existe.
    Y si existe, la elimina y la vuelve a crear.
    """
    if mssparkutils.fs.exists(f"file:{output}"):
        mssparkutils.fs.rm(f"file:{output}", True)

    mssparkutils.fs.mkdirs(f"file:{output}")

StatementMeta(, 0adcee9e-0c7b-4301-8d32-7386cc155280, 18, Finished, Available, Finished)

In [17]:
# Obteniendo información del modelo semántico

print("Obteniendo los metadatos del modelo semántico...")
semantic_model = get_semantic_model(WORKSPACE_ID, DATASET_ID)

# Generando los ficheros HTML

print(f"Preparando la carpeta para guardar las páginas HTML: {HTML_FOLDER}")
prepare_output_folder(HTML_FOLDER)

environment = Environment(loader=DictLoader(jinja_templates))

html_file_model = f"{HTML_FOLDER}/model.html"
print(f"Generando la página HTML del modelo: {html_file_model}")
generate_model_page(environment, "model.html", html_file_model, semantic_model)

for table_id in semantic_model["tables"]["ID"]:
    html_file_table = f"{HTML_FOLDER}/table_{table_id}.html"
    print(
        f"Generando la página HTML para la tabla del modelo con el ID {table_id} en: {html_file_table}"
    )
    generate_table_page(
        table_id,
        environment,
        "table.html",
        html_file_table,
        semantic_model,
    )

print(f"Ya todo está listo en la carpeta: {HTML_FOLDER}")

StatementMeta(, 0adcee9e-0c7b-4301-8d32-7386cc155280, 19, Finished, Available, Finished)

Obteniendo los metadatos del modelo semántico...
Preparando la carpeta para guardar las páginas HTML: /lakehouse/default/Files/semdoc_html/1e5db40c-f9c2-4aa5-91b0-7c27f047fd4a
Generando la página HTML del modelo: /lakehouse/default/Files/semdoc_html/1e5db40c-f9c2-4aa5-91b0-7c27f047fd4a/model.html
Generando la página HTML para la tabla del modelo con el ID 10 en: /lakehouse/default/Files/semdoc_html/1e5db40c-f9c2-4aa5-91b0-7c27f047fd4a/table_10.html
Generando la página HTML para la tabla del modelo con el ID 16 en: /lakehouse/default/Files/semdoc_html/1e5db40c-f9c2-4aa5-91b0-7c27f047fd4a/table_16.html
Generando la página HTML para la tabla del modelo con el ID 19 en: /lakehouse/default/Files/semdoc_html/1e5db40c-f9c2-4aa5-91b0-7c27f047fd4a/table_19.html
Generando la página HTML para la tabla del modelo con el ID 22 en: /lakehouse/default/Files/semdoc_html/1e5db40c-f9c2-4aa5-91b0-7c27f047fd4a/table_22.html
Generando la página HTML para la tabla del modelo con el ID 25 en: /lakehouse/defa